# Complete Text-to-SQL Tree Orchestration with All 4 Agents

This notebook demonstrates a complete text-to-SQL tree orchestration process using all 4 specialized agents:
- **QueryAnalyzerAgent**: Analyzes user queries and creates query trees
- **SchemaLinkerAgent**: Links query intents to database schema
- **SQLGeneratorAgent**: Generates SQL from linked schema
- **SQLEvaluatorAgent**: Executes and evaluates SQL results

## Key Features

1. **Structured Memory Management**: Uses KeyValueMemory with specialized managers
2. **Query Tree Architecture**: All agents operate on nodes in a query tree
3. **Automatic SQL Execution**: SQLEvaluator automatically executes SQL
4. **Iterative Refinement**: Coordinator can retry if SQL is incorrect
5. **Complete Traceability**: Full visibility into the tree orchestration process

In [ ]:
import os
import sys
import asyncio
import logging
from pathlib import Path
from typing import Dict, Any, List, Optional
from dotenv import load_dotenv

sys.path.append('../src')
load_dotenv()

# Important: For running this notebook, ensure OPENAI_API_KEY is set
# You can run: source ../.env && export OPENAI_API_KEY
# Or set it here directly (not recommended for production)
if not os.getenv("OPENAI_API_KEY"):
    print("WARNING: OPENAI_API_KEY not found in environment")
    print("Run: source ../.env && export OPENAI_API_KEY")
else:
    print("OPENAI_API_KEY found in environment")

# Set up logging - disable most debug messages but keep LLM calls
logging.basicConfig(level=logging.INFO, 
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# Keep autogen_core.events at INFO to see LLM calls
logging.getLogger('autogen_core.events').setLevel(logging.INFO)

# Disable other noisy loggers
logging.getLogger('autogen_core').setLevel(logging.WARNING)
logging.getLogger('autogen_agentchat').setLevel(logging.WARNING)
logging.getLogger('httpx').setLevel(logging.WARNING)
logging.getLogger('openai').setLevel(logging.WARNING)
logging.getLogger('httpcore').setLevel(logging.WARNING)
logging.getLogger('httpcore.connection').setLevel(logging.WARNING)
logging.getLogger('httpcore.http11').setLevel(logging.WARNING)

In [ ]:
# Custom formatter for LLM calls
import json

class LLMCallFormatter(logging.Formatter):
    """Custom formatter that makes LLM call logs more readable"""
    
    def format(self, record):
        if record.name == 'autogen_core.events' and hasattr(record, 'msg'):
            try:
                # Try to parse as JSON
                if isinstance(record.msg, str) and record.msg.startswith('{'):
                    data = json.loads(record.msg)
                    if data.get('type') == 'LLMCall':
                        # Extract key information
                        source = data.get('source', 'unknown')
                        prompt_tokens = data.get('prompt_tokens', 0)
                        completion_tokens = data.get('completion_tokens', 0)
                        model = data.get('model', 'unknown')
                        
                        # Create formatted message
                        return f"🤖 LLM Call [{source}] using {model}: {prompt_tokens} prompt + {completion_tokens} completion = {prompt_tokens + completion_tokens} total tokens"
            except:
                pass
        
        # Default formatting for other messages
        return super().format(record)

# Apply custom formatter to autogen_core.events logger
events_logger = logging.getLogger('autogen_core.events')
for handler in events_logger.handlers:
    handler.setFormatter(LLMCallFormatter('%(message)s'))

# Also apply to root logger handlers
for handler in logging.root.handlers:
    handler.setFormatter(LLMCallFormatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s'))

print("✓ Custom LLM call formatter applied")

# Memory and managers
from keyvalue_memory import KeyValueMemory
from task_context_manager import TaskContextManager
from query_tree_manager import QueryTreeManager
from database_schema_manager import DatabaseSchemaManager
from node_history_manager import NodeHistoryManager

# Schema reader
from schema_reader import SchemaReader

# All 4 agents + task status checker
from query_analyzer_agent import QueryAnalyzerAgent
from schema_linker_agent import SchemaLinkerAgent
from sql_generator_agent import SQLGeneratorAgent
from sql_evaluator_agent import SQLEvaluatorAgent
from task_status_checker import TaskStatusChecker

# Memory types
from memory_content_types import (
    TaskContext, QueryNode, NodeStatus, TaskStatus,
    QueryMapping, TableMapping, ColumnMapping, JoinMapping,
    TableSchema, ColumnInfo, ExecutionResult
)

# AutoGen components
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_ext.models.openai import OpenAIChatCompletionClient

In [ ]:
# Memory and managers
from keyvalue_memory import KeyValueMemory
from task_context_manager import TaskContextManager
from query_tree_manager import QueryTreeManager
from database_schema_manager import DatabaseSchemaManager
from node_history_manager import NodeHistoryManager

# Schema reader
from schema_reader import SchemaReader

# All 4 agents + task status checker
from query_analyzer_agent import QueryAnalyzerAgent
from schema_linker_agent import SchemaLinkerAgent
from sql_generator_agent import SQLGeneratorAgent
from sql_evaluator_agent import SQLEvaluatorAgent
from task_status_checker import TaskStatusChecker

# Memory types
from memory_content_types import (
    TaskContext, QueryNode, NodeStatus, TaskStatus,
    QueryMapping, TableMapping, ColumnMapping, JoinMapping,
    TableSchema, ColumnInfo, ExecutionResult
)

# AutoGen components
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_ext.models.openai import OpenAIChatCompletionClient

In [3]:
# Initialize shared memory
memory = KeyValueMemory()

# Initialize managers
task_manager = TaskContextManager(memory)
tree_manager = QueryTreeManager(memory)
schema_manager = DatabaseSchemaManager(memory)
history_manager = NodeHistoryManager(memory)

print("Initialized memory and managers")

Initialized memory and managers


## 3. Load Database Schema

In [4]:
# Database configuration
data_path = "/home/norman/work/text-to-sql/MAC-SQL/data/bird"
tables_json_path = Path(data_path) / "dev_tables.json"
db_name = "california_schools"

# Initialize task
task_id = "workflow_demo_001"
# Test query
test_query = "What is the highest eligible free rate for K-12 students in schools located in Alameda County?"
print(f"Processing query: {test_query}")
print("-" * 80)

await task_manager.initialize(task_id, test_query, db_name)

# Load schema using SchemaReader
schema_reader = SchemaReader(
    data_path=data_path,
    tables_json_path=str(tables_json_path),
    dataset_name="bird",
    lazy=False
)

# Load schema into memory
await schema_manager.load_from_schema_reader(schema_reader, db_name)

# Get schema summary
summary = await schema_manager.get_schema_summary()
print(f"Loaded database '{db_name}' schema:")
print(f"  Tables: {summary['table_count']}")
print(f"  Total columns: {summary['total_columns']}")
print(f"  Foreign keys: {summary['total_foreign_keys']}")

2025-05-25 17:54:42,777 - TaskContextManager - INFO - Initialized task context for task workflow_demo_001


Processing query: What is the highest eligible free rate for K-12 students in schools located in Alameda County?
--------------------------------------------------------------------------------
load json file from /home/norman/work/text-to-sql/MAC-SQL/data/bird/dev_tables.json

Loading all database info...
Found 11 databases in bird dataset


2025-05-25 17:54:55,229 - DatabaseSchemaManager - INFO - Initialized empty database schema
2025-05-25 17:54:55,230 - DatabaseSchemaManager - INFO - Added table 'frpm' to schema
2025-05-25 17:54:55,230 - DatabaseSchemaManager - INFO - Added table 'satscores' to schema
2025-05-25 17:54:55,231 - DatabaseSchemaManager - INFO - Added table 'schools' to schema
2025-05-25 17:54:55,231 - DatabaseSchemaManager - INFO - Loaded schema for database 'california_schools' with 3 tables


Loaded database 'california_schools' schema:
  Tables: 3
  Total columns: 89
  Foreign keys: 2


# LLM configuration
llm_config = {
    "model_name": "gpt-4o",
    "temperature": 0.1,
    "timeout": 60
}

# Initialize all agents
query_analyzer = QueryAnalyzerAgent(memory, llm_config)
schema_linker = SchemaLinkerAgent(memory, llm_config)
sql_generator = SQLGeneratorAgent(memory, llm_config)
sql_evaluator = SQLEvaluatorAgent(memory, llm_config)

# Initialize TaskStatusChecker (no LLM config needed)
task_status_checker = TaskStatusChecker(memory)

print("Initialized all agents including TaskStatusChecker")

In [ ]:
# LLM configuration
llm_config = {
    "model_name": "gpt-4o",
    "temperature": 0.1,
    "timeout": 60
}

# Initialize all agents
query_analyzer = QueryAnalyzerAgent(memory, llm_config)
schema_linker = SchemaLinkerAgent(memory, llm_config)
sql_generator = SQLGeneratorAgent(memory, llm_config)
sql_evaluator = SQLEvaluatorAgent(memory, llm_config)
task_status_checker = TaskStatusChecker(memory, llm_config)

print("Initialized all agents including TaskStatusChecker")

# Initialize OpenAI client for coordinator
coordinator_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    temperature=0.1,
    timeout=120,
    api_key=os.getenv("OPENAI_API_KEY")
)

# Create coordinator agent with TaskStatusChecker
coordinator = AssistantAgent(
    name="coordinator",
    system_message="""You coordinate a text-to-SQL workflow using specialized agents and a task status checker.

Your agents are:
1. query_analyzer - Analyzes user queries and creates query trees
2. schema_linker - Links query intent to database schema
3. sql_generator - Generates SQL from linked schema
4. sql_evaluator - Executes and evaluates SQL results for single nodes
5. task_status_checker - Checks overall tree status and determines next actions

WORKFLOW PROCESS:

1. Start by calling query_analyzer with the user's query

2. For each node processing cycle:
   - If current node has no mapping → call schema_linker
   - If current node has no SQL → call sql_generator
   - If current node has SQL but not executed → call sql_evaluator
   - After sql_evaluator completes → ALWAYS call task_status_checker

3. CRITICAL: Parsing task_status_checker response
   The task_status_checker will include an ACTION line like:
   - ACTION: PROCESS NODE [node_id]: [description]
   - ACTION: RETRY NODE [node_id]: [description]
   - ACTION: TASK COMPLETE: All nodes have been successfully processed
   - ACTION: ERROR: [description]
   
   You MUST:
   - Look for the ACTION line in the response
   - If ACTION contains "PROCESS NODE" → Continue processing that node
   - If ACTION contains "RETRY NODE" → Retry that node
   - If ACTION contains "TASK COMPLETE" → Terminate with final answer
   - If ACTION contains "ERROR" → Handle the error

HOW TO CALL AGENTS:
- query_analyzer: Call with the user's query directly
- Other agents: Call with simple task descriptions:
  - "Link query to database schema"
  - "Generate SQL query"
  - "Analyze SQL execution results"
  - "Check task status and determine next action"

CRITICAL RULES:
- ALWAYS call task_status_checker after sql_evaluator
- Parse the ACTION line from task_status_checker's response
- ONLY terminate when ACTION line says "TASK COMPLETE"
- Do NOT terminate based on individual node results
- Do NOT assume completion - let task_status_checker decide

TERMINATION:
- Search for "ACTION: TASK COMPLETE" in task_status_checker's response
- ONLY say "TERMINATE" when you find this exact phrase
- Before terminating, summarize ALL results from ALL nodes processed

For complex queries:
- Children nodes must be processed before parent nodes
- Each child must have good quality SQL before moving to next
- Parent nodes combine results from children
- Continue until task_status_checker says "TASK COMPLETE" """,
    model_client=coordinator_client,
    tools=[
        query_analyzer.get_tool(), 
        schema_linker.get_tool(), 
        sql_generator.get_tool(), 
        sql_evaluator.get_tool(),
        task_status_checker.get_tool()
    ]
)

In [ ]:
# Initialize OpenAI client for coordinator
coordinator_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    temperature=0.1,
    timeout=120,
    api_key=os.getenv("OPENAI_API_KEY")
)

# Create coordinator agent with TaskStatusChecker
coordinator = AssistantAgent(
    name="coordinator",
    system_message="""You coordinate a text-to-SQL workflow using specialized agents and a task status checker.

Your agents are:
1. query_analyzer - Analyzes user queries and creates query trees
2. schema_linker - Links query intent to database schema
3. sql_generator - Generates SQL from linked schema
4. sql_evaluator - Executes and evaluates SQL results for single nodes
5. task_status_checker - Checks overall tree status and determines next actions

WORKFLOW PROCESS:

1. Start by calling query_analyzer with the user's query

2. For each node processing cycle:
   - If current node has no mapping → call schema_linker
   - If current node has no SQL → call sql_generator
   - If current node has SQL but not executed → call sql_evaluator
   - After sql_evaluator completes → ALWAYS call task_status_checker

3. CRITICAL: Parsing task_status_checker response
   The task_status_checker will include an ACTION line like:
   - ACTION: PROCESS NODE [node_id]: [description]
   - ACTION: RETRY NODE [node_id]: [description]
   - ACTION: TASK COMPLETE: All nodes have been successfully processed
   - ACTION: ERROR: [description]
   
   You MUST:
   - Look for the ACTION line in the response
   - If ACTION contains "PROCESS NODE" → Continue processing that node
   - If ACTION contains "RETRY NODE" → Retry that node
   - If ACTION contains "TASK COMPLETE" → Terminate with final answer
   - If ACTION contains "ERROR" → Handle the error

HOW TO CALL AGENTS:
- query_analyzer: Call with the user's query directly
- Other agents: Call with simple task descriptions:
  - "Link query to database schema"
  - "Generate SQL query"
  - "Analyze SQL execution results"
  - "Check task status and determine next action"

CRITICAL RULES:
- ALWAYS call task_status_checker after sql_evaluator
- Parse the ACTION line from task_status_checker's response
- ONLY terminate when ACTION line says "TASK COMPLETE"
- Do NOT terminate based on individual node results
- Do NOT assume completion - let task_status_checker decide

TERMINATION:
- Search for "ACTION: TASK COMPLETE" in task_status_checker's response
- ONLY say "TERMINATE" when you find this exact phrase
- Before terminating, summarize ALL results from ALL nodes processed

For complex queries:
- Children nodes must be processed before parent nodes
- Each child must have good quality SQL before moving to next
- Parent nodes combine results from children
- Continue until task_status_checker says "TASK COMPLETE" """,
    model_client=coordinator_client,
    tools=[query_analyzer.get_tool(), schema_linker.get_tool(), 
           sql_generator.get_tool(), sql_evaluator.get_tool(),
           task_status_checker.get_tool()]
)

## 6. Define Helper Functions

### Helper to Display Full Memory Contents

In [7]:
async def display_memory_contents():
    """Display full memory contents in a structured format"""
    print("\n" + "="*80)
    print("FULL MEMORY CONTENTS")
    print("="*80)
    
    # Get all memory data
    memory_data = await memory.show_all(format="json")
    
    if not memory_data:
        print("Memory is empty")
        return
    
    for key, value in memory_data.items():
        print(f"\n[{key}]")
        print("-" * 40)
        
        if isinstance(value["value"], dict):
            # Pretty print dictionaries
            import json
            print(json.dumps(value["value"], indent=2))
        elif isinstance(value["value"], list):
            # Pretty print lists
            import json
            print(json.dumps(value["value"], indent=2))
        else:
            # Print raw value
            print(value["value"])
    
    print("\n" + "="*80)
    print(f"Total keys in memory: {len(memory_data)}")
    print("="*80)

async def display_query_tree():
    """Display the current query tree structure"""
    tree = await tree_manager.get_tree()
    if not tree or "nodes" not in tree:
        print("No query tree found")
        return
    
    print("\n" + "="*60)
    print("QUERY TREE STRUCTURE")
    print("="*60)
    
    for node_id, node_data in tree["nodes"].items():
        print(f"\nNode: {node_id}")
        print(f"  Intent: {node_data.get('intent', 'N/A')}")
        print(f"  Status: {node_data.get('status', 'N/A')}")
        
        # Show mapping if available
        if 'mapping' in node_data and node_data['mapping']:
            mapping = node_data['mapping']
            if mapping.get('tables'):
                tables = [t['name'] for t in mapping['tables']]
                print(f"  Tables: {', '.join(tables)}")
            if mapping.get('columns'):
                cols = [f"{c['table']}.{c['column']}" for c in mapping['columns']]
                print(f"  Columns: {', '.join(cols[:3])}..." if len(cols) > 3 else f"  Columns: {', '.join(cols)}")
        
        # Show SQL if available
        if 'sql' in node_data and node_data['sql']:
            sql_preview = node_data['sql'].strip().replace('\n', ' ')[:100]
            print(f"  SQL: {sql_preview}..." if len(sql_preview) == 100 else f"  SQL: {sql_preview}")
        
        # Show execution result if available
        if 'executionResult' in node_data and node_data['executionResult']:
            result = node_data['executionResult']
            print(f"  Execution: {result.get('rowCount', 0)} rows")
            if result.get('error'):
                print(f"  Error: {result['error']}")

async def display_final_results():
    """Display the final SQL and execution results"""
    tree = await tree_manager.get_tree()
    if not tree or "nodes" not in tree:
        print("No results found")
        return
    
    print("\n" + "="*60)
    print("FINAL RESULTS")
    print("="*60)
    
    # Find nodes with SQL
    for node_id, node_data in tree["nodes"].items():
        if 'sql' in node_data and node_data['sql']:
            print(f"\nNode: {node_id}")
            print(f"Intent: {node_data.get('intent', 'N/A')}")
            print(f"\nSQL:\n{node_data['sql']}")
            
            if 'executionResult' in node_data and node_data['executionResult']:
                result = node_data['executionResult']
                print(f"\nExecution Result:")
                print(f"  Rows returned: {result.get('rowCount', 0)}")
                
                if result.get('data') and len(result['data']) > 0:
                    print(f"\nSample data (first 5 rows):")
                    for i, row in enumerate(result['data'][:5]):
                        print(f"  {row}")
                
                # Check for analysis
                analysis_key = f"node_{node_id}_analysis"
                analysis = await memory.get(analysis_key)
                if analysis:
                    print(f"\nEvaluation:")
                    print(f"  Answers intent: {analysis.get('answers_intent', 'N/A')}")
                    print(f"  Result quality: {analysis.get('result_quality', 'N/A')}")
                    print(f"  Summary: {analysis.get('result_summary', 'N/A')}")

## 7. Run the Workflow

In [ ]:
# Create a team with termination condition
termination_condition = TextMentionTermination("TERMINATE")
team = RoundRobinGroupChat(
    participants=[coordinator],
    termination_condition=termination_condition
)

# Run the workflow
stream = team.run_stream(task=test_query)

In [ ]:
# Stream and display messages with status checker tracking
message_count = 0
step_count = 0
last_agent_called = None

async for message in stream:
    # Track coordinator steps
    if hasattr(message, 'source') and message.source == 'coordinator':
        step_count += 1
        print(f"\n[Step {step_count}] Coordinator:")
        if hasattr(message, 'content'):
            if isinstance(message.content, str):
                # Check for termination
                if "TERMINATE" in message.content:
                    print("  → Workflow complete, terminating...")
                    print(f"\nFinal answer: {message.content.replace('TERMINATE', '').strip()}")
                else:
                    # Show coordinator reasoning (abbreviated)
                    preview = message.content[:150] + "..." if len(message.content) > 150 else message.content
                    print(f"  Reasoning: {preview}")
            elif isinstance(message.content, list) and len(message.content) > 0:
                # Tool calls
                for tool_call in message.content:
                    if hasattr(tool_call, 'name'):
                        last_agent_called = tool_call.name
                        print(f"  → Calling {tool_call.name}")
    
    # Track task_status_checker responses
    elif hasattr(message, 'source') and last_agent_called == 'task_status_checker':
        if hasattr(message, 'content') and isinstance(message.content, str):
            # Extract and display ACTION
            import re
            action_match = re.search(r'ACTION:\s*(.+?)(?:\n|$)', message.content)
            if action_match:
                action = action_match.group(1).strip()
                print(f"  📍 Task Status: {action}")
    
    message_count += 1

print("\n" + "="*80)
print("WORKFLOW COMPLETE")
print(f"Total messages: {message_count}, Coordinator steps: {step_count}")
print("="*80)

## 8. Display Query Tree and Results

In [10]:
# Display the query tree structure
await display_query_tree()


QUERY TREE STRUCTURE

Node: node_1748210101.323748_root
  Intent: Find the highest percentage of students eligible for free meals in K-12 schools located in Alameda County.
  Status: sql_generated
  Tables: frpm
  Columns: frpm.County Name, frpm.Percent (%) Eligible Free (K-12)
  SQL: -- SQL query to find the highest percentage of students eligible for free meals in K-12 schools loca...

Node: node_1748210101.32443_1
  Intent: Filter schools located in Alameda County
  Status: created
  Tables: schools

Node: node_1748210101.324725_2
  Intent: Find the highest eligible free rate for K-12 students
  Status: created
  Tables: frpm


## 9. Display Full Memory Contents

This shows all data stored in memory after the tree orchestration completes.

In [11]:
# Display full memory contents
await display_memory_contents()


FULL MEMORY CONTENTS

[node_node_1748210101.32443_1_analysis]
----------------------------------------
{
  "answers_intent": "partially",
  "result_quality": "acceptable",
  "result_summary": "The results should show a list of schools located in Alameda County.",
  "confidence_score": 0.5,
  "issues": [
    {
      "type": "logic",
      "description": "The SQL query is missing, so it's unclear if the filtering logic for Alameda County is correctly implemented.",
      "severity": "high"
    },
    {
      "type": "data_quality",
      "description": "Without the results, it's impossible to assess if there are any NULL values or unexpected data entries.",
      "severity": "medium"
    }
  ],
  "suggestions": [
    "Provide the SQL query and execution results to enable a detailed analysis.",
    "Ensure the query includes a WHERE clause filtering by Alameda County."
  ]
}

[execution_analysis]
----------------------------------------
{
  "answers_intent": "partially",
  "result_qualit

## 10. Test with Complex Query

Let's test the workflow with a complex query that creates multiple nodes to ensure TaskStatusChecker properly manages the tree progression.

In [ ]:
# Clear memory for a fresh test
await memory.clear()

# Complex query that should create multiple nodes
complex_query = "Find the top 5 counties by average SAT scores, including the number of schools and average free lunch rate"

print(f"Testing complex query: {complex_query}")
print("-" * 80)

# Reinitialize
task_id = "complex_workflow_demo"
await task_manager.initialize(task_id, complex_query, db_name)
await schema_manager.load_from_schema_reader(schema_reader, db_name)

# Create fresh team
complex_team = RoundRobinGroupChat(
    participants=[coordinator],
    termination_condition=termination_condition
)

# Run the complex workflow
print("\nStarting complex workflow with TaskStatusChecker...")
complex_stream = complex_team.run_stream(task=complex_query)

In [ ]:
# Process complex workflow with detailed tracking
step_count = 0
node_processing = {}
task_status_actions = []

async for message in complex_stream:
    if hasattr(message, 'source') and message.source == 'coordinator':
        step_count += 1
        
        if hasattr(message, 'content'):
            if isinstance(message.content, list):
                # Track agent calls
                for tool_call in message.content:
                    if hasattr(tool_call, 'name'):
                        agent_name = tool_call.name
                        print(f"\n[Step {step_count}] → {agent_name}")
                        
                        # Track which node is being processed
                        if agent_name in ['schema_linker', 'sql_generator', 'sql_evaluator']:
                            current_node = await memory.get("current_node_id")
                            if current_node:
                                if current_node not in node_processing:
                                    node_processing[current_node] = []
                                node_processing[current_node].append(agent_name)
                                print(f"    Processing node: {current_node}")
                        
                        elif agent_name == 'task_status_checker':
                            print(f"    Checking overall task status...")
            
            elif isinstance(message.content, str) and "TERMINATE" in message.content:
                print(f"\n[Step {step_count}] ✅ WORKFLOW COMPLETE")
                break
    
    # Capture task status checker actions
    elif hasattr(message, 'source') and hasattr(message, 'content'):
        if isinstance(message.content, str) and "ACTION:" in message.content:
            import re
            action_match = re.search(r'ACTION:\s*(.+?)(?:\n|$)', message.content)
            if action_match:
                action = action_match.group(1).strip()
                task_status_actions.append(action)
                print(f"    📍 Status: {action}")

print("\n" + "="*80)
print("COMPLEX WORKFLOW SUMMARY")
print("="*80)
print(f"Total steps: {step_count}")
print(f"Nodes processed: {len(node_processing)}")
print(f"\nNode processing history:")
for node, agents in node_processing.items():
    print(f"  {node}: {' → '.join(agents)}")

print(f"\nTask status actions:")
for i, action in enumerate(task_status_actions, 1):
    print(f"  {i}. {action}")